In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz
# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetXLS.xlsx
/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv
/kaggle/input/indian-food-101/indian_food.csv


In [2]:
# Read the Indian Food 101 Dataset
df_101 = pd.read_csv('/kaggle/input/indian-food-101/indian_food.csv')
# Display Top 5 rows
df_101.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [3]:
# Read 6000 Indian Food Recipes Dataset
df_6000 = pd.read_csv('/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv')
# Display top 5 rows
df_6000.head()

,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
0,1,Masala Karela Recipe,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...","6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...","To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...
1,2,टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...,Spicy Tomato Rice (Recipe),"2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...","2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,टमाटर पुलियोगरे बनाने के लिए सबसे पहले टमाटर क...,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...","1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...","To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...","500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,5,आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 बड़ा चमच्च चना दाल,1 बड़ा चमच्च सफ़ेद उरद दाल,2...","1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,आंध्रा स्टाइल आलम पचड़ी बनाने के लिए सबसे पहले ...,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...


In [4]:
# Total number of recipes by state from Indian Food 101 dataset
df_101.state.value_counts()

Gujarat            35
Punjab             32
Maharashtra        30
West Bengal        24
-1                 24
Assam              21
Tamil Nadu         20
Andhra Pradesh     10
Uttar Pradesh       9
Kerala              8
Odisha              7
Karnataka           6
Rajasthan           6
Telangana           5
Goa                 3
Bihar               3
Jammu & Kashmir     2
Madhya Pradesh      2
Manipur             2
Uttarakhand         1
NCT of Delhi        1
Nagaland            1
Tripura             1
Haryana             1
Chhattisgarh        1
Name: state, dtype: int64

In [5]:
# Extract only recipes belong to Tamil Nadu

df_101[df_101['state'] == 'Tamil Nadu']

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
49,Pongal,"Rice, jaggery, cashews, ghee",vegetarian,5,20,sweet,dessert,Tamil Nadu,South
53,Unni Appam,"Rice flour, banana, jaggery, coconut",vegetarian,10,20,sweet,dessert,Tamil Nadu,South
127,Currivepillai sadam,"Chana dal, urad dal, fresh coconut, sesame see...",vegetarian,10,20,spicy,main course,Tamil Nadu,South
129,Idiappam,"Rice flour, hot water, grated coconut",vegetarian,120,30,spicy,snack,Tamil Nadu,South
132,Kaara kozhambu,"Sesame oil, drumstick, tamarind paste, sambar ...",vegetarian,-1,-1,spicy,main course,Tamil Nadu,South
133,Keerai kootu,"Moong dal, chana dal, spinach, urad dal, cocon...",vegetarian,20,30,spicy,main course,Tamil Nadu,South
134,Keerai masiyal,"Urad dal, curry leaves, sugar, mustard seeds, ...",vegetarian,-1,-1,spicy,main course,Tamil Nadu,South
135,Keerai sadam,"Greens, tomato, mustard seeds, fenugreek seeds",vegetarian,10,20,spicy,main course,Tamil Nadu,South
136,Keerai poriyal,"Amaranth leaves, split urad dal, mustard seeds...",vegetarian,20,30,spicy,main course,Tamil Nadu,South
138,Kootu,"Chana dal, urad dal, potato, beans, peas",vegetarian,10,15,spicy,main course,Tamil Nadu,South


# Let's choose a dish from 101 dataset and extract different varieties of the same from 6000 dataset

## We all love varieties in our faood :-) 

In [6]:
# Use process to extract top 20 matches for the recipe pongal
possibilities = process.extract("pongal", df_6000['TranslatedRecipeName'], limit=20, scorer=fuzz.partial_ratio)

In [9]:
# Get matches with partial ratio as 100
# Partial rtaio compare subsections of the string
extracted_matches = [possible for possible in possibilities if possible[1] == 100]
extracted_matches

[('Pudina Khara Pongal Recipe (Rice and Lentils Cooked With Mint Leaves & Mild Spices)',
  100,
  5),
 ('Foxtail Millet Pongal Recipe', 100, 509),
 ('Oats Ven Pongal Recipe', 100, 512),
 ('Kalkandu (Kallusakkare) Pongal Recipe | Rice Pudding with Rock Candy',
  100,
  1173),
 ('Pudina Khara Pongal Recipe - Pudina Khara Pongal Recipe', 100, 1452),
 ('Oats Sugar Pongal Recipe', 100, 1904),
 ('Chakkra Pongali Recipe (Andhra Style Sweet Pongal)', 100, 2722),
 ('Barnyard Millet Sweet Pongal Recipe (Kuthiraivali Sakkarai Pongal)',
  100,
  3251),
 ('Oats And Spinach Pongal Recipe', 100, 3925),
 (' Rava Pongal (Recipe In Hindi)', 100, 4047),
 ('Up to Rice Pongal Recipe -Kodo Millet Pongal', 100, 4583),
 (' Oats Ven Pongal (Recipe In Hindi)', 100, 4761),
 ('Sweet Pongal With Thai Jasmine Rice And Palm Sugar Recipe', 100, 5824),
 ('Sakkare Millet Pongal Recipe', 100, 5846),
 ('Milagu Ven Pongal Recipe Made With Figaro Olive Oil', 100, 6687)]

In [22]:
# Extract only the recipe name from the result
get_recipe_name = [extracted_matches[i][0] for i in range(len(extracted_matches))]

In [23]:
# Get recipes of the extracted matches
df_6000[df_6000['TranslatedRecipeName'].isin(get_recipe_name)]

,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
5,6,Pudina Khara Pongal Recipe (Rice and Lentils C...,Pudina Khara Pongal Recipe (Rice and Lentils C...,"1 cup Rice - soaked for 20 minutes,1/2 cup Yel...","1 cup Rice - soaked for 20 minutes,1/2 cup Yel...",10,20,30,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making Pudina Khara Pongal Recipe, wa...","To begin making Pudina Khara Pongal Recipe, wa...",http://www.archanaskitchen.com/pudina-khara-po...
509,551,Foxtail Millet Pongal Recipe,Foxtail Millet Pongal Recipe,"1/2 cup Foxtail Millet,1/2 cup Yellow Moong Da...","1/2 cup Foxtail Millet,1/2 cup Yellow Moong Da...",70,35,105,15,South Indian Recipes,South Indian Breakfast,Vegetarian,"To prepare Foxtail Millet Pongal, first dry ro...","To prepare Foxtail Millet Pongal, first dry ro...",https://www.archanaskitchen.com/foxtail-millet...
512,554,Oats Ven Pongal Recipe,Oats Ven Pongal Recipe,"1 tablespoon Ghee,1 cup Instant Oats (Oatmeal)...","1 tablespoon Ghee,1 cup Instant Oats (Oatmeal)...",10,30,40,4,South Indian Recipes,South Indian Breakfast,Vegetarian,To begin making the Oats Ven Pongal Recipe we ...,To begin making the Oats Ven Pongal Recipe we ...,http://www.archanaskitchen.com/oats-ven-pongal...
1173,1308,Kalkandu (Kallusakkare) Pongal Recipe | Rice P...,Kalkandu (Kallusakkare) Pongal Recipe | Rice P...,"1/4 cup Rice,1 cup Milk - + 1 tablespoon,1/4 c...","1/4 cup Rice,1 cup Milk - + 1 tablespoon,1/4 c...",20,20,40,2,South Indian Recipes,Dessert,Vegetarian,To prepare Kalkandu (Kallusakkare) Pongal Reci...,To prepare Kalkandu (Kallusakkare) Pongal Reci...,http://www.archanaskitchen.com/kalkandu-kallus...
1452,1627,पुदीना खरा पोंगल रेसिपी - Pudina Khara Pongal ...,Pudina Khara Pongal Recipe - Pudina Khara Pong...,"1 कप चावल - 20 मिनट के लिए भिगो ले,1/2 कप पिली...","1 cup rice - soaked for 20 minutes, 1/2 cup ye...",10,20,30,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,पुदीना खरा पोंगल बनाने के लिए सबसे पहले चावल औ...,पुदीना खरा पोंगल बनाने के लिए सबसे पहले चावल औ...,https://www.archanaskitchen.com/pudina-khara-p...
1904,2188,Oats Sakkarai Pongal Recipe,Oats Sugar Pongal Recipe,"1 tablespoon Ghee,1 cup Instant Oats (Oatmeal)...","1 tablespoon Ghee,1 cup Instant Oats (Oatmeal)...",15,30,45,3,South Indian Recipes,Dessert,Vegetarian,To begin making the Oats Sakkarai Pongal Recip...,To begin making the Oats Sakkarai Pongal Recip...,https://www.archanaskitchen.com/oats-sakkarai-...
2722,3271,Chakkra Pongali Recipe (Andhra Style Sweet Pon...,Chakkra Pongali Recipe (Andhra Style Sweet Pon...,"1/2 cup Rice,1/3 cup Yellow Moong Dal (Split),...","1/2 cup Rice,1/3 cup Yellow Moong Dal (Split),...",5,30,35,2,Andhra,Dessert,Vegetarian,To begin making Chakkra Pongali Recipe (Andhra...,To begin making Chakkra Pongali Recipe (Andhra...,https://www.archanaskitchen.com/chakkra-pongal...
3251,4010,Barnyard Millet Sweet Pongal Recipe (Kuthiraiv...,Barnyard Millet Sweet Pongal Recipe (Kuthiraiv...,"3/4 cup Barnyard Millet,1/4 cup Yellow Moong D...","3/4 cup Barnyard Millet,1/4 cup Yellow Moong D...",20,40,60,4,South Indian Recipes,South Indian Breakfast,Vegetarian,To begin making the Barnyard Millet Sweet Pong...,To begin making the Barnyard Millet Sweet Pong...,http://www.archanaskitchen.com/barnyard-millet...
3925,5114,Oats And Spinach Pongal Recipe,Oats And Spinach Pongal Recipe,"3/4 cup Instant Oats (Oatmeal),1/2 cup Yellow ...","3/4 cup Instant Oats (Oatmeal),1/2 cup Yellow ...",10,25,35,4,Tamil Nadu,South Indian Breakfast,High Protein Vegetarian,To begin making the Oats And Spinach Pongal re...,To begin making the Oats And Spinach Pongal re...,https://www.archanaskitchen.com/oats-and-spina...
4047,5312,रवा पोंगल रेसिपी - Rava Pongal (Recipe In Hindi),Rava Pongal (Recipe In Hindi),"1/2 कप पिली मूंग दाल,1 कप सूजी,2 बड़े चमच्च घी,...","1/2 कप पिली म